# Toy Paxos

> A playground to test and learn how Paxos work

### Overview

This Notebook implements a basic Paxos alogrithm and provides a PaxosNode that one can use to simulate various scenarious of the consensus protocol and try to understand how each node makes decision and reach consensus.

#### How to use
First create a PaxosNode with a name and Quorum size 
```
node = PaxosNode("Node", quorum=3)
another_node = PaxosNode("Another_Node", quorum=3)
```

Now initialze a new proposal - A node can re-initialize any time
```
node.init_new_proposal()
```

Request promise from other nodes
```
node.promise_request(another_node)

Output:

PROMISE Request from Node(p: 1) to Another_Node(acc_p: None)
    > [ACC] Promise(1, last_acepted()), Node(promisers: 1, quorum_size: 3, promiser_acc_max: None)

Interpreataion:
* Node sent a promsie request to Another_Node
* Node sent proposal_id p=1
* Annother_node had no preveious accepted proposal acc_p = None
* The proposal was [ACC] Accepted
* Promise was accpted for proposal id 1, another_node had not last_accepted() value in promise response
* Final state of Node is: 1 promiseer, need 2 more to reach quorum of size 3, promiser_acc_max is None as no proposer sent back accepted value otherwise traks max value accpted by any promiser.
```
Request Accept from another node
```
node.accept_request(another_node)

Output:

ACCEPT  Request from Node(p: 1, val: Apple, promiser_acc_max: Mango) to Another_Node(acc_p: 1)
    > [ACC] Accept(1, Mango), Node(acceptors: 3, quorum_size: 3, choice: Mango)

Interpreatation:
* Node sent accept request to Another_Node
* Node's accept request was agains proosal id p_id =1
* Node wants to propose 'Apple'
* Node's promisers have accepted a value 'Mango' i.e promiser_acc_max
* Another ndoe has promised for id acc_p = 1
* Another node [ACC] Accepted the request
* Request was accepted for id 1 and value 'Mango' sent by Node
* Node's final stat is it got 3 accpetors until now, since quorum size is thre it has reached quorum and hence chose the value it requested to acceppt (Mango) as the chosen value.
```

   

In [9]:
##
# Accept :  This model stores the result of an `accept_request`
# * proposal_id : The proposal_id accpted by the node
# * value: The value that was accoted by the node
# * is_accepted: True if the current request was accepted, False if previous accepted data is returned
##
class Accept(object):
    def __init__(self, proposal_id: int, value: str, is_accepted: bool):
        self.proposal_id = proposal_id
        self.value = value
        self.is_accepted = is_accepted

    def __str__(self) -> str:
        status = "[ACC]" if self.is_accepted else "[REJ]"
        return f"{status} Accept({self.proposal_id}, {self.value})"

In [10]:
##
# Promise :  This model stores the result of a `promise_request`
# * proposal_id : The proposal_id promnised by the node
# * last_accepted: last accepted value (if any) by the node
# * is_accepted: True if the current request was accepted, False if previous promised data is returned
##
class Promise(object):
    def __init__(self, proposal_id: int, is_accepted: bool, last_accepted: Accept = None):
        self.proposal_id = proposal_id
        self.is_accepted = is_accepted
        self.last_accepted = last_accepted
        
    def __str__(self) -> str:
        status = "[ACC]" if self.is_accepted else "[REJ]"        
        last_accepted_info = "()"
        if self.last_accepted is not None:
            last_accepted_info = f"(val: {self.last_accepted.value}, p: {self.last_accepted.proposal_id})"
        return f"{status} Promise({self.proposal_id}, last_acepted{last_accepted_info})"

In [74]:
from __future__ import annotations

class PaxosNode(object):
    proposal_id_counter = 0
    
    def __init__(self, node_id: str, quorum: int):
        self.node_id: str = node_id
        self.quorum: int = quorum
        self.last_proposal_id_accepted: int = None
        self.last_accepted_value: str = None
        self.last_accepted_p_id: int = None
        self._proposal_id = None

    # Creates a new proposal_id to attempt to get a value chosen
    def init_new_proposal(self, value: str):
        self._value_to_propose = value
        self._proposal_id = self._generateProposalId()
        self._promisers = dict()
        self._promiser_last_accpted: Accept = None
        self._acceptors = dict()
        print(f"{self.node_id} generated a new proposal id {self._proposal_id} for value : {value}")

    ##
    # This method tries to get a value chosen by completely execting both phases of paxos
    # It first gets promises from prodived nodes
    # If a quorum is reached on promise, 
    # sends accepts to each of them choosing if there were any accepted value from proposers
    # If a quorim is reached on accept, the value accepted by acceptors is the 'chosen' value
    ##
    def propose(self, value: str, *nodes: PaxosNodes) -> str:
        while len(set(nodes)) >= self.quorum:
            self.init_new_proposal(value)
            for node in nodes:
                self.promise_request(node)
            if len(self._promisers) < self.quorum:
                print ("Unable to reach quorum on promises")
                continue # Try with a new proposal id
            for node in nodes:
                self.accept_request(node)
            if len(self._acceptors) < self.quorum:
                print ("Unable to reach quorum on accepts")
                continue # Try new proposal
            choice = list(self._acceptors.values())[0].value # A chouce is made by now
            print(f"{self.node_id} chose : {choice}")
            break

    # Try to get promise from given node
    def promise_request(self, node: PaxosNode):
        if self._proposal_id is None:
            print("Proposal not initiated, Cannot send promise request")
            return
        
        node_prev_p_id = node.last_proposal_id_accepted
        promise: Promise = node._promise(self._proposal_id)
        
        if promise.is_accepted:
            self._promisers[node.node_id] = promise
            if promise.last_accepted != None:
                if self._promiser_last_accpted == None or self._promiser_last_accpted.proposal_id < promise.last_accepted.proposal_id:
                    self._promiser_last_accpted = promise.last_accepted
        else:
            self._promisers.pop(node.node_id, None)
        
        promiser_acc_max = self._promiser_last_accpted.value if self._promiser_last_accpted != None else None
        print(f"PROMISE Request from {self.node_id}(p: {self._proposal_id}) to {node.node_id}(acc_p: {node_prev_p_id})")
        print(f"    > {promise}, {self.node_id}(promisers: {len(self._promisers)}, \
quorum_size: {self.quorum}, promiser_acc_max: {promiser_acc_max})")

    # If a node has got promise quorum, it can request accept from the given node
    def accept_request(self, node: PaxosNode) -> str:
        if self._proposal_id is None or len(self._promisers) < self.quorum:
            print("No promise quorum - cannot send accept request")
            return

        node_prev_p_id = node.last_proposal_id_accepted
        max_promised_value = self._promiser_last_accpted.value if self._promiser_last_accpted is not None else None
        value_to_propose = max_promised_value if max_promised_value is not None else self._value_to_propose
        accept = node._accept(self._proposal_id, value_to_propose)
        if accept.is_accepted:
            self._acceptors[node.node_id] = accept
        else:
            self._acceptors.pop(node.node_id, None)

        has_accept_quorum = len(self._acceptors) >= self.quorum
        choice = value_to_propose if has_accept_quorum else None
        
        print(f"ACCEPT  Request from {self.node_id}(p: {self._proposal_id}, val: {self._value_to_propose}, \
promiser_acc_max: {max_promised_value}) to {node.node_id}(acc_p: {node_prev_p_id})")
        print(f"    > {accept}, {self.node_id}(acceptors: {len(self._acceptors)}, quorum_size: {self.quorum}, choice: {choice})")

    # Accept's logic to respond to promise request
    def _promise(self, proposal_id: int) -> Promise:
        is_accepted = False
        if self.last_proposal_id_accepted == None or self.last_proposal_id_accepted <= proposal_id:
            is_accepted = True
            self.last_proposal_id_accepted = proposal_id
        last_accepted = Accept(self.last_accepted_p_id, self.last_accepted_value, True) if self.last_accepted_p_id != None else None
        return Promise(self.last_proposal_id_accepted, is_accepted, last_accepted)

    # Acceptors logic to respond to a Accept request
    def _accept(self, proposal_id: int, value: str) -> Accept:
        is_accepted = False
        if self.last_proposal_id_accepted == None or self.last_proposal_id_accepted <= proposal_id:
            is_accepted = True
            self.last_proposal_id_accepted = proposal_id
            self.last_accepted_p_id = proposal_id
            self.last_accepted_value = value
        return Accept(self.last_accepted_p_id, self.last_accepted_value, is_accepted)

    # Function to generate unique proposal id higer than anythig generated until now
    def _generateProposalId(self):
        PaxosNode.proposal_id_counter += 1
        return PaxosNode.proposal_id_counter

### Case 1: Proposer B tries to propose a new value after Proposer A has chosen a value

In [75]:
# 5 friends, Quorum = 3, 2 Movies!
amy = PaxosNode('Amy', 3)
ben = PaxosNode('Ben', 3)
cal = PaxosNode('Cal', 3)
dan = PaxosNode('Dan', 3)
eva = PaxosNode('Eva', 3)

amy.init_new_proposal("Openhimer")
amy.promise_request(amy)
amy.promise_request(ben)
amy.promise_request(cal)
print("== Amy reached pronise quorun ===\n")
amy.accept_request(amy)
amy.accept_request(ben)
amy.accept_request(cal) 
print("== Amy chosen a value ===\n")

eva.init_new_proposal("Barbie")
eva.promise_request(cal)
eva.promise_request(dan)
eva.promise_request(eva) # Should get a quorum
print("== Eva reached promise quorun ===\n")

eva.accept_request(cal)
eva.accept_request(dan)
eva.accept_request(eva)
print("== Eva learnt a chosen value ===\n")

Amy generated a new proposal id 1 for value : Openhimer
PROMISE Request from Amy(p: 1) to Amy(acc_p: None)
    > [ACC] Promise(1, last_acepted()), Amy(promisers: 1, quorum_size: 3, promiser_acc_max: None)
PROMISE Request from Amy(p: 1) to Ben(acc_p: None)
    > [ACC] Promise(1, last_acepted()), Amy(promisers: 2, quorum_size: 3, promiser_acc_max: None)
PROMISE Request from Amy(p: 1) to Cal(acc_p: None)
    > [ACC] Promise(1, last_acepted()), Amy(promisers: 3, quorum_size: 3, promiser_acc_max: None)
== Amy reached pronise quorun ===

ACCEPT  Request from Amy(p: 1, val: Openhimer, promiser_acc_max: None) to Amy(acc_p: 1)
    > [ACC] Accept(1, Openhimer), Amy(acceptors: 1, quorum_size: 3, choice: None)
ACCEPT  Request from Amy(p: 1, val: Openhimer, promiser_acc_max: None) to Ben(acc_p: 1)
    > [ACC] Accept(1, Openhimer), Amy(acceptors: 2, quorum_size: 3, choice: None)
ACCEPT  Request from Amy(p: 1, val: Openhimer, promiser_acc_max: None) to Cal(acc_p: 1)
    > [ACC] Accept(1, Openhimer), 

### Case 2: Proposer B gets a promise quorum but one of the node has already accepted value by Proposer A

In [76]:
amy = PaxosNode('Amy', 3)
ben = PaxosNode('Ben', 3)
cal = PaxosNode('Cal', 3)
dan = PaxosNode('Dan', 3)
eva = PaxosNode('Eva', 3)

amy.init_new_proposal("Openhimer")
amy.promise_request(amy)
amy.promise_request(ben)
amy.promise_request(cal)
print("== Amy reached pronise quorun ===\n")
amy.accept_request(cal)
print("Amy takes a break\n")

eva.init_new_proposal("Barbie")
eva.promise_request(cal)
eva.promise_request(dan)
eva.promise_request(eva) # Should get a quorum
print("== Eva reached promise quorun ===\n")

eva.accept_request(cal)
eva.accept_request(dan)
eva.accept_request(eva)
print("== Eva learnt a chosen value ===\n")

amy.accept_request(amy)
amy.accept_request(ben) 
print("== Amy chosen a value ===\n")

Amy generated a new proposal id 3 for value : Openhimer
PROMISE Request from Amy(p: 3) to Amy(acc_p: None)
    > [ACC] Promise(3, last_acepted()), Amy(promisers: 1, quorum_size: 3, promiser_acc_max: None)
PROMISE Request from Amy(p: 3) to Ben(acc_p: None)
    > [ACC] Promise(3, last_acepted()), Amy(promisers: 2, quorum_size: 3, promiser_acc_max: None)
PROMISE Request from Amy(p: 3) to Cal(acc_p: None)
    > [ACC] Promise(3, last_acepted()), Amy(promisers: 3, quorum_size: 3, promiser_acc_max: None)
== Amy reached pronise quorun ===

ACCEPT  Request from Amy(p: 3, val: Openhimer, promiser_acc_max: None) to Cal(acc_p: 3)
    > [ACC] Accept(3, Openhimer), Amy(acceptors: 1, quorum_size: 3, choice: None)
Amy takes a break

Eva generated a new proposal id 4 for value : Barbie
PROMISE Request from Eva(p: 4) to Cal(acc_p: 3)
    > [ACC] Promise(4, last_acepted(val: Openhimer, p: 3)), Eva(promisers: 1, quorum_size: 3, promiser_acc_max: Openhimer)
PROMISE Request from Eva(p: 4) to Dan(acc_p: None

In [79]:
### Case 3: Proposer B gets a promise quorum and no promised nodes have accepted value from Proposer A

In [77]:
amy = PaxosNode('Amy', 3)
ben = PaxosNode('Ben', 3)
cal = PaxosNode('Cal', 3)
dan = PaxosNode('Dan', 3)
eva = PaxosNode('Eva', 3)

amy.init_new_proposal("Openhimer")
amy.promise_request(amy)
amy.promise_request(ben)
amy.promise_request(cal)
print("== Amy reached pronise quorun ===\n")
amy.accept_request(ben)
print("Amy takes a break\n")

eva.init_new_proposal("Barbie")
eva.promise_request(cal)
eva.promise_request(dan)
eva.promise_request(eva) # Should get a quorum
print("== Eva reached promise quorun ===\n")

eva.accept_request(cal)
eva.accept_request(dan)
eva.accept_request(eva)
print("== Eva learnt a chosen value ===\n")

amy.accept_request(amy)
amy.accept_request(cal) 
print("== Amy gets a rej ===\n")

Amy generated a new proposal id 5 for value : Openhimer
PROMISE Request from Amy(p: 5) to Amy(acc_p: None)
    > [ACC] Promise(5, last_acepted()), Amy(promisers: 1, quorum_size: 3, promiser_acc_max: None)
PROMISE Request from Amy(p: 5) to Ben(acc_p: None)
    > [ACC] Promise(5, last_acepted()), Amy(promisers: 2, quorum_size: 3, promiser_acc_max: None)
PROMISE Request from Amy(p: 5) to Cal(acc_p: None)
    > [ACC] Promise(5, last_acepted()), Amy(promisers: 3, quorum_size: 3, promiser_acc_max: None)
== Amy reached pronise quorun ===

ACCEPT  Request from Amy(p: 5, val: Openhimer, promiser_acc_max: None) to Ben(acc_p: 5)
    > [ACC] Accept(5, Openhimer), Amy(acceptors: 1, quorum_size: 3, choice: None)
Amy takes a break

Eva generated a new proposal id 6 for value : Barbie
PROMISE Request from Eva(p: 6) to Cal(acc_p: 5)
    > [ACC] Promise(6, last_acepted()), Eva(promisers: 1, quorum_size: 3, promiser_acc_max: None)
PROMISE Request from Eva(p: 6) to Dan(acc_p: None)
    > [ACC] Promise(6, 